In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [ ]:
df_panel   = pd.read_csv('единицы_без_протестов_test.csv')
df_treated = pd.read_csv('протесты_test.csv')

df_panel['group'] = 0   # Контрольные муниципалитеты
df_treated['group'] = 1   # Протестные муниципалитеты

# 3. Объединяем вертикально (concatenate)
df_panel = pd.concat([df_panel, df_treated], ignore_index=True)

df_panel = df_panel[['year', 'municipality', 'oktmo',
                     'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',
                     'region_name', 'group']].drop_duplicates(subset=['year','municipality','oktmo', 'group'])




# df_panel.to_csv('panel_all.csv', index=False)
df_panel = df_panel[df_panel['year'].isin([2009, 2012])].reset_index(drop=True)
df_panel['post'] = (df_panel['year'] == 2012).astype(int)

df_panel = df_panel.rename(columns={
    'Жилье (1000 м2)': 'Жилье',
    'Дороги (%)': 'Дороги',
    'Канализация (1 м)': 'Канализация',
    'Водопровод (1 метр)': 'Водопровод'
})



def run_did(data, outcome):
    # Удаляем строки с пропусками по outcome
    d = data.dropna(subset=[outcome, 'group', 'post']).copy()
    formula = f'{outcome} ~ group + post + group:post + C(region_name)'
    model = smf.ols(formula, data=d).fit(cov_type='HC1')
    return model

outcomes = ['Жилье', 'Дороги', 'Канализация', 'Водопровод']
results = {o: run_did(df_panel, o) for o in outcomes}

print("\nКоэффициенты взаимодействия (group:post) и p-value:")
coef_rows = []
for o, m in results.items():
    coef = m.params.get('group:post', 0)
    pval = m.pvalues.get('group:post', 1)
    coef_rows.append([o, coef, pval])
    print(f"{o}: {coef:.4f} (p-value: {pval:.3f})")
    # print(m.summary())





Коэффициенты взаимодействия (group:post) и p-value:
Жилье: -17.2191 (p-value: 0.640)
Дороги: 20.5082 (p-value: 0.000)
Канализация: -195.1638 (p-value: 0.569)
Водопровод: -3422.9987 (p-value: 0.288)


In [12]:
df_panel   = pd.read_csv('единицы_без_протестов_2010t3.csv')
df_treated = pd.read_csv('протесты_2010t3.csv')


df_panel['group'] = 0   # Контрольные муниципалитеты
df_treated['group'] = 1   # Протестные муниципалитеты

# 3. Объединяем вертикально (concatenate)
df_panel = pd.concat([df_panel, df_treated], ignore_index=True)

df_panel = df_panel[['year', 'municipality', 'oktmo',
                     'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',
                     'region_name', 'group']].drop_duplicates(subset=['year','municipality','oktmo'])




# df_panel.to_csv('panel_all3.csv', index=False)
df_panel = df_panel[df_panel['year'].isin([2009, 2013])].reset_index(drop=True)
df_panel['post'] = (df_panel['year'] == 2013).astype(int)

df_panel = df_panel.rename(columns={
    'Жилье (1000 м2)': 'Жилье',
    'Дороги (%)': 'Дороги',
    'Канализация (1 м)': 'Канализация',
    'Водопровод (1 метр)': 'Водопровод'
})



def run_did(data, outcome):
    # Удаляем строки с пропусками по outcome
    d = data.dropna(subset=[outcome, 'group', 'post']).copy()
    formula = f'{outcome} ~ group + post + group:post + C(region_name)'
    model = smf.ols(formula, data=d).fit(cov_type='HC1')
    return model

outcomes = ['Жилье', 'Дороги', 'Канализация', 'Водопровод']
results = {o: run_did(df_panel, o) for o in outcomes}

print("\nКоэффициенты взаимодействия (group:post) и p-value:")
coef_rows = []
for o, m in results.items():
    coef = m.params.get('group:post', 0)
    pval = m.pvalues.get('group:post', 1)
    coef_rows.append([o, coef, pval])
    print(f"{o}: {coef:.4f} (p-value: {pval:.3f})")
    # print(m.summary())





Коэффициенты взаимодействия (group:post) и p-value:
Жилье: 50.9519 (p-value: 0.405)
Дороги: 23.5417 (p-value: 0.000)
Канализация: -358.1879 (p-value: 0.302)
Водопровод: -4097.3927 (p-value: 0.132)


In [ ]:

datasets = ['2010t.csv', '2010tM.csv', '2010tM2.csv', '2010t+.csv']

# Годы для анализа
years_combinations = [
    [2008, 2011],
    [2008, 2012],
    [2008, 2013]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest',
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name',
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event',
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name',
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2010t.csv

Годы: 2008 -- 2011
Коэффициенты взаимодействия (group:post):
Жилье: -1.25 (p-value: 0.995)
                            OLS Regression Results                            
Dep. Variable:                  Жилье   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     3.166
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           1.05e-09
Time:                        15:28:10   Log-Likelihood:                -2050.7
No. Observations:                 256   AIC:                             4221.
Df Residuals:                     196   BIC:                             4434.
Df Model:                          59                                         
Covariance Type:                  HC1                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]


In [8]:

datasets = ['2012t.csv', '2012tM.csv', '2012tM2.csv', '2012t+.csv']

# Годы для анализа
years_combinations = [
    [2010, 2014],
    [2010, 2015],
    [2010, 2016]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest'
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event'
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2012t.csv

Годы: 2010 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: 14.92 (p-value: 0.446)
Дороги: 3.03 (p-value: 0.571)
Канализация: -1556.25 (p-value: 0.278)
Водопровод: -358.26 (p-value: 0.832)

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -2.57 (p-value: 0.880)
Дороги: 12.70 (p-value: 0.018)
Канализация: -1830.26 (p-value: 0.200)
Водопровод: -1362.01 (p-value: 0.390)

Годы: 2010 -- 2016
Коэффициенты взаимодействия (group:post):
Жилье: 88.05 (p-value: 0.410)
Дороги: 10.55 (p-value: 0.048)
Канализация: -1810.34 (p-value: 0.207)
Водопровод: -794.89 (p-value: 0.632)

2012tM.csv

Годы: 2010 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: 10.71 (p-value: 0.544)
Дороги: 7.38 (p-value: 0.182)
Канализация: -1813.56 (p-value: 0.207)
Водопровод: -924.32 (p-value: 0.586)

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -7.32 (p-value: 0.651)
Дороги: 12.44 (p-value: 0.025)
Канализация: -2124.28 (p-value: 0.139)
Водопровод: -1343.16

In [9]:

datasets = ['2010t5_2.csv', '2010t5M_2.csv', '2010t5M2_2.csv', '2010t5+_2.csv']

# Годы для анализа
years_combinations = [
    [2008, 2013],
    [2008, 2014],
    [2008, 2015]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest'
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event'
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2010t5_2.csv

Годы: 2008 -- 2013
Коэффициенты взаимодействия (group:post):
Жилье: -3.00 (p-value: 0.894)
Дороги: 13.07 (p-value: 0.004)
Канализация: -60.34 (p-value: 0.739)
Водопровод: 1342.35 (p-value: 0.093)

Годы: 2008 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: -4.46 (p-value: 0.842)
Дороги: 14.36 (p-value: 0.002)
Канализация: 2.08 (p-value: 0.992)
Водопровод: 760.60 (p-value: 0.343)

Годы: 2008 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -22.47 (p-value: 0.288)
Дороги: 14.59 (p-value: 0.002)
Канализация: -103.22 (p-value: 0.557)
Водопровод: 414.15 (p-value: 0.570)

2010t5M_2.csv

Годы: 2008 -- 2013
Коэффициенты взаимодействия (group:post):
Жилье: -12.75 (p-value: 0.530)
Дороги: 12.23 (p-value: 0.008)
Канализация: -192.30 (p-value: 0.233)
Водопровод: 736.95 (p-value: 0.383)

Годы: 2008 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: -11.97 (p-value: 0.551)
Дороги: 12.83 (p-value: 0.007)
Канализация: 69.37 (p-value: 0.686)
Водопровод: 443.09 (p-va

In [10]:

datasets = ['2010t6.csv', '2010t6M.csv', '2010t6M2.csv', '2010t6+.csv']

# Годы для анализа
years_combinations = [
    [2010, 2015],
    [2010, 2016],
    [2010, 2017]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest'
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event'
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2010t6.csv

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -9.18 (p-value: 0.511)
Дороги: 10.30 (p-value: 0.033)
Канализация: -1794.05 (p-value: 0.143)
Водопровод: -1035.59 (p-value: 0.445)

Годы: 2010 -- 2016
Коэффициенты взаимодействия (group:post):
Жилье: 73.23 (p-value: 0.419)
Дороги: 10.15 (p-value: 0.040)
Канализация: -1803.77 (p-value: 0.144)
Водопровод: -484.67 (p-value: 0.734)

Годы: 2010 -- 2017
Коэффициенты взаимодействия (group:post):
Жилье: -21.16 (p-value: 0.108)
Дороги: 19.06 (p-value: 0.000)
Канализация: -1855.73 (p-value: 0.131)
Водопровод: -1481.53 (p-value: 0.278)

2010t6M.csv

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -5.38 (p-value: 0.704)
Дороги: 11.98 (p-value: 0.017)
Канализация: -1834.51 (p-value: 0.133)
Водопровод: -1495.40 (p-value: 0.303)

Годы: 2010 -- 2016
Коэффициенты взаимодействия (group:post):
Жилье: 71.23 (p-value: 0.465)
Дороги: 11.75 (p-value: 0.020)
Канализация: -1730.47 (p-value: 0.159)
Водопровод: -

In [11]:

datasets = ['2010t4.csv', '2010t4М.csv', '2010t4М2.csv', '2010t4+.csv']

# Годы для анализа
years_combinations = [
    [2008, 2013],
    [2008, 2014],
    [2008, 2015]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest'
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event'
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2010t4.csv

Годы: 2008 -- 2013
Коэффициенты взаимодействия (group:post):
Жилье: 28.76 (p-value: 0.466)
Дороги: -4.53 (p-value: 0.599)
Канализация: -489.57 (p-value: 0.465)
Водопровод: 487.93 (p-value: 0.777)

Годы: 2008 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: 17.65 (p-value: 0.683)
Дороги: -3.93 (p-value: 0.644)
Канализация: -239.19 (p-value: 0.717)
Водопровод: 967.45 (p-value: 0.580)

Годы: 2008 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -125.20 (p-value: 0.014)
Дороги: -7.69 (p-value: 0.360)
Канализация: -793.70 (p-value: 0.198)
Водопровод: -357.64 (p-value: 0.828)

2010t4М.csv

Годы: 2008 -- 2013
Коэффициенты взаимодействия (group:post):
Жилье: 42.74 (p-value: 0.566)
Дороги: -3.35 (p-value: 0.717)
Канализация: -221.02 (p-value: 0.752)
Водопровод: -910.31 (p-value: 0.548)

Годы: 2008 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: 31.68 (p-value: 0.661)
Дороги: -1.22 (p-value: 0.891)
Канализация: 19.42 (p-value: 0.976)
Водопровод: -1892.30 (p-

In [12]:

datasets = ['2010t3.csv', '2010t3М.csv', '2010t3М2.csv', '2010t3+.csv']

# Годы для анализа
years_combinations = [
    [2010, 2014],
    [2010, 2015],
    [2010, 2016]
]

for dataset in datasets:
    print(f"\n{dataset}")
    print("=" * 50)
    
    # Загрузка данных
    df = pd.read_csv(dataset)
    
    # Подготовка данных
    protest = df[[
        'year', 
        'municipality_protest', 
        'oktmo_protest', 
        'Жилье (1000 м2)_protest', 
        'Дороги (%)_protest', 
        'Канализация (1 м)_protest', 
        'Водопровод (1 метр)_protest',
        'region_name_protest'
    ]]

    protest.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    protest['group'] = 1  

    event = df[[
        'year', 
        'municipality_event', 
        'oktmo_event',
        'Жилье (1000 м2)_event', 
        'Дороги (%)_event', 
        'Канализация (1 м)_event', 
        'Водопровод (1 метр)_event',
        'region_name_event'
    ]]
    event.columns = [
        'year', 
        'municipality', 
        'oktmo', 
        'Жилье', 
        'Дороги', 
        'Канализация', 
        'Водопровод',
        'region_name'
    ]
    event['group'] = 0  

    combined = pd.concat([protest, event], ignore_index=True)

    def run_did(data, outcome, years):
        data_filtered = data[data['year'].isin(years)].copy()
        data_filtered['post'] = (data_filtered['year'] == years[1]).astype(int)
        data_filtered = data_filtered.dropna(subset=[outcome, 'group', 'post'])
        
        formula = f'{outcome} ~ group + post + group:post + C(region_name) + C(year)'
        model = smf.ols(formula, data=data_filtered).fit(cov_type='HC1')
        return model

    for years in years_combinations:
        print(f"\nГоды: {years[0]} -- {years[1]}")
        
        results = {}
        for outcome in ['Жилье', 'Дороги', 'Канализация', 'Водопровод']:
            model = run_did(combined, outcome, years)
            results[outcome] = model

        print("Коэффициенты взаимодействия (group:post):")
        for outcome, model in results.items():
            coef = model.params.get('group:post', 0)
            pval = model.pvalues.get('group:post', 1)
            print(f"{outcome}: {coef:.2f} (p-value: {pval:.3f})")


2010t3.csv

Годы: 2010 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: -13.34 (p-value: 0.790)
Дороги: -2.45 (p-value: 0.777)
Канализация: 3033.80 (p-value: 0.321)
Водопровод: -2970.69 (p-value: 0.133)

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -184.83 (p-value: 0.004)
Дороги: -2.21 (p-value: 0.808)
Канализация: 2935.21 (p-value: 0.331)
Водопровод: -2595.79 (p-value: 0.112)

Годы: 2010 -- 2016
Коэффициенты взаимодействия (group:post):
Жилье: -190.34 (p-value: 0.003)
Дороги: -1.02 (p-value: 0.913)
Канализация: 3060.89 (p-value: 0.313)
Водопровод: -2934.25 (p-value: 0.127)

2010t3М.csv

Годы: 2010 -- 2014
Коэффициенты взаимодействия (group:post):
Жилье: -14.53 (p-value: 0.853)
Дороги: 7.06 (p-value: 0.426)
Канализация: 4.04 (p-value: 0.994)
Водопровод: -1946.37 (p-value: 0.313)

Годы: 2010 -- 2015
Коэффициенты взаимодействия (group:post):
Жилье: -233.69 (p-value: 0.000)
Дороги: 5.88 (p-value: 0.522)
Канализация: -345.48 (p-value: 0.323)
Водопровод: -23